# 2章

In [13]:
import sys, numpy, pandas, polars, matplotlib, seaborn, sklearn
print(f"✅ Python {sys.version.split()[0]} | NumPy {numpy.__version__} | Pandas {pandas.__version__} | Polars {polars.__version__} | Matplotlib {matplotlib.__version__} | Seaborn {seaborn.__version__} | Scikit-learn {sklearn.__version__}")

✅ Python 3.13.3 | NumPy 2.3.2 | Pandas 2.3.1 | Polars 1.31.0 | Matplotlib 3.10.3 | Seaborn 0.13.2 | Scikit-learn 1.7.1


In [14]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

ふざけたエラーに遭遇。
解決策...[自分用メモ：OpenAIのライブラリ使うと「TypeError: Client.__init__() got an unexpected keyword argument 'proxies'」が出る - Qiita](https://qiita.com/hatman621221/items/fafaf4ac0e906616e8bd)

In [15]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "こんにちは私はジョンといいます。"},
    ],
)

print(response.to_json(indent=2))

{
  "id": "chatcmpl-BzT0NAXlZI5XiNzu1Xdb3X97k5NM3",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "こんにちは、ジョンさん！お会いできて嬉しいです。今日はどんなことをお話ししましょうか？",
        "refusal": null,
        "role": "assistant",
        "annotations": []
      }
    }
  ],
  "created": 1753989203,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_34a54ae93c",
  "usage": {
    "completion_tokens": 27,
    "prompt_tokens": 25,
    "total_tokens": 52,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  }
}


## Function Calling

## 概要

利用可能な関数を伝えておいて、LLMに関数を使いたいという判断をさせる機能。

In [16]:
import json

def get_current_weather(location, unit="fahrenheit"):
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

In [17]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters":{
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

In [18]:
from openai import OpenAI

client = OpenAI()

messages = [
    {"role": "user", "content": "東京の天気はどうですか？"},
]

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

print(response.to_json(indent=2))

{
  "id": "chatcmpl-BzT0Pt5GczpcNhcu2V1FxnIBeXm1Q",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "tool_calls": [
          {
            "id": "call_XR3uXIWv9RahIp2Aq7SN6INp",
            "function": {
              "arguments": "{\"location\":\"Tokyo\"}",
              "name": "get_current_weather"
            },
            "type": "function"
          }
        ],
        "annotations": []
      }
    }
  ],
  "created": 1753989205,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_a288987b44",
  "usage": {
    "completion_tokens": 15,
    "prompt_tokens": 81,
    "total_tokens": 96,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0

今までは以下のように、contentに回答が入っていた。

```json
{
  "id": "chatcmpl-ByN3qPlS6TnWflRVxXv2Eoy5w2vxy",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "こんにちは、ジョンさん！お会いできて嬉しいです。どんなことについてお話ししましょうか？",
        "refusal": null,
        "role": "assistant",
        "annotations": []
      }
    }
  ],
  ...
}
```

しかし今回はcontentが空で、代わりにtool_calls要素が追加され、そこにやりたいことが書いてある。

```json
{
  "id": "chatcmpl-ByNql8AqNI3x3KSQECqXHMJsMQOU3",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "tool_calls": [
          {
            "id": "call_ijDcqnaT1gTexDUHPS2NdNmi",
            "function": {
              "arguments": "{\"location\":\"Tokyo, Japan\",\"unit\":\"celsius\"}",
              "name": "get_current_weather"
            },
            "type": "function"
          }
        ],
        "annotations": []
      }
    }
  ],
...
      "rejected_prediction_tokens": 0
    }
  }
}
```

In [19]:
response_message = response.choices[0].message
messages.append(response_message.to_dict())

In [20]:
print(response_message)
print(messages)

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_XR3uXIWv9RahIp2Aq7SN6INp', function=Function(arguments='{"location":"Tokyo"}', name='get_current_weather'), type='function')], annotations=[])
[{'role': 'user', 'content': '東京の天気はどうですか？'}, {'content': None, 'refusal': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_XR3uXIWv9RahIp2Aq7SN6INp', 'function': {'arguments': '{"location":"Tokyo"}', 'name': 'get_current_weather'}, 'type': 'function'}], 'annotations': []}]


In [21]:
available_functions = {
    "get_current_weather": get_current_weather,
}

# 使いたい関数は複数ある可能性があるので、ループ
for tool_call in response_message.tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        location = function_args.get("location"),
        unit = function_args.get("unit"),
    )

    # 書籍のミス、このコードがない。
    print(function_response)

    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response
        }
    )

{"location": "Tokyo", "temperature": "10", "unit": null}


In [22]:
print(json.dumps(messages, ensure_ascii=False, indent=2))

[
  {
    "role": "user",
    "content": "東京の天気はどうですか？"
  },
  {
    "content": null,
    "refusal": null,
    "role": "assistant",
    "tool_calls": [
      {
        "id": "call_XR3uXIWv9RahIp2Aq7SN6INp",
        "function": {
          "arguments": "{\"location\":\"Tokyo\"}",
          "name": "get_current_weather"
        },
        "type": "function"
      }
    ],
    "annotations": []
  },
  {
    "tool_call_id": "call_XR3uXIWv9RahIp2Aq7SN6INp",
    "role": "tool",
    "name": "get_current_weather",
    "content": "{\"location\": \"Tokyo\", \"temperature\": \"10\", \"unit\": null}"
  }
]


In [23]:
second_response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
)

print(second_response.to_json(indent=2))

{
  "id": "chatcmpl-BzT0QxSrjC36M3aMWrd74PecBRCiO",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "現在、東京の気温は10度です。詳細な天気情報は、具体的な日付や天気の状態に応じて異なる場合がありますので、ローカルの天気予報サービスで最新情報を確認してください。",
        "refusal": null,
        "role": "assistant",
        "annotations": []
      }
    }
  ],
  "created": 1753989206,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_07871e2ad8",
  "usage": {
    "completion_tokens": 54,
    "prompt_tokens": 57,
    "total_tokens": 111,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  }
}


## tool_choice

Chat Completions APIのパラメータには`tool_choice`がある。

`= none`では、関数を呼び出さない。
`= auto`では、いい感じに呼んでくれる。

関数名を指定して、実行する関数を指定できる。

## 二章のまとめ

GPT 4o, GPT 4o-miniをAPIで呼びたいときは、`Chat Completions API`を使用する。

`Chat Completions API`はステートレスであるため、会話履歴を踏まえて応答してほしいときは、全会話履歴をリクエストに持たせる。

料金はモデルの種類と入出力のトークン数で決まる。
使用したトークン数はレスポンスに含まれる。

function_calling機能では、LLMにローカルの関数を使うか判断させることが出来る。
AIエージェントの実装に使われることが多い。